# Library Load

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys
import sktime
import tqdm as tq
import matplotlib
import seaborn as sns
import sklearn as skl
import random
import os
from tqdm import tqdm
import tqdm as tq

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings(action = 'ignore')

# set seed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) #

# Data load

In [2]:
train_df = pd.read_csv('/Users/idoyun/Desktop/데이터 분석 경진대회/진행중/2023 전력사용량 예측 AI 경진대회/datafile/train.csv')
test_df = pd.read_csv('/Users/idoyun/Desktop/데이터 분석 경진대회/진행중/2023 전력사용량 예측 AI 경진대회/datafile/test.csv')

In [3]:
building_info = pd.read_csv('/Users/idoyun/Desktop/데이터 분석 경진대회/진행중/2023 전력사용량 예측 AI 경진대회/datafile/building_info.csv')

In [4]:
train_df = train_df.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
train_df.drop('num_date_time', axis = 1, inplace=True)

In [5]:
building_info = building_info.rename(columns={
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'
})

In [6]:
translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '대학교': 'University',
    '데이터센터': 'Data Center',
    '백화점및아울렛': 'Department Store and Outlet',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    '지식산업센터': 'Knowledge Industry Center',
    '할인마트': 'Discount Mart',
    '호텔및리조트': 'Hotel and Resort'
}

building_info['building_type'] = building_info['building_type'].replace(translation_dict)

In [7]:
train_df = pd.merge(train_df, building_info, on='building_number', how='left')

In [8]:
# solar_power_capacity, ess_capacity, pcs_capacity 대부분 결측자료 drop
train_df = train_df.drop(['solar_power_capacity', 'ess_capacity', 'pcs_capacity'], axis=1)
train_df = train_df.drop(['solar_radiation', 'sunshine'], axis=1)

In [9]:
columns_to_fill = ['rainfall', 'windspeed', 'humidity']
train_df[columns_to_fill] = train_df[columns_to_fill].fillna(0)

In [10]:
# 시계열 변수 생성
date = pd.to_datetime(train_df.date_time)
train_df['hour'] = date.dt.hour
train_df['day'] = date.dt.weekday
train_df['month'] = date.dt.month
train_df['week'] = date.dt.weekofyear

In [11]:
# building_type 변수를 수치형으로 변환하여 새로운 열 생성
building_type_mapping = {
    'Apartment': 1,
    'Commercial': 2,
    'Data Center': 3,
    'Department Store and Outlet': 4,
    'Discount Mart': 5,
    'Hospital': 6,
    'Hotel and Resort': 7,
    'Knowledge Industry Center': 8,
    'Other Buildings': 9,
    'Public': 10,
    'Research Institute': 11,
    'University': 12
}
train_df['building_type_numeric'] = train_df['building_type'].map(building_type_mapping)

In [12]:
### 공휴일 변수 추가
train_df['holiday'] = train_df['day'].apply(lambda x: 1 if x >= 6 else 0)

In [13]:
## https://dacon.io/competitions/official/235680/codeshare/2366?page=1&dtype=recent
# 순환적 성격의 시간 변수 생성
train_df['sin_time'] = np.sin(2*np.pi*train_df.hour/24)
train_df['cos_time'] = np.cos(2*np.pi*train_df.hour/24)

In [14]:
## https://dacon.io/competitions/official/235736/codeshare/2743?page=1&dtype=recent
# 상대습도 변수 생성
train_df['THI'] = 9/5*train_df['temperature'] - 0.55*(1-train_df['humidity']/100)*(9/5*train_df['humidity']-26)+32

In [15]:
# Cooling Degree Hour 변수 생성
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

cdhs = np.array([])
for num in range(1,101,1):
    temp = train_df[train_df['building_number'] == num]
    cdh = CDH(temp['temperature'].values)
    cdhs = np.concatenate([cdhs, cdh])
train_df['CDH'] = cdhs

In [16]:
# feature과 target data 분리
x_train = train_df.drop(columns = ['date_time','power_consumption','building_type',
                                  'total_area','cooling_area',
                                   'hour','building_type_numeric'])
y_train = train_df['power_consumption']

In [22]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size = 0.2)

In [23]:
x_train.head()

,building_number,temperature,rainfall,windspeed,humidity,day,month,week,holiday,sin_time,cos_time,THI,CDH
133244,66,28.5,0.0,1.9,68.0,0,6,26,0,-0.866025,0.500000,66.3336,36.6
194488,96,24.6,0.2,12.1,96.0,2,6,26,0,-0.866025,-0.500000,73.0504,-21.2
651,1,27.4,0.0,3.1,77.0,1,6,26,0,0.707107,0.707107,67.0761,13.9
109258,54,20.2,8.0,4.0,100.0,0,7,29,0,0.500000,-0.866025,68.3600,-25.4
72710,36,30.6,0.0,2.6,65.0,0,7,30,0,-0.500000,-0.866025,69.5625,7.8


feature :  
building_number, temperature, rainfall, windspeed, humidity, day, month, week, building_type_numeric,  
holiday, sin_time, cos_time, THI, CDH

target : power consumption

# Random Forest

In [19]:
rf_model = RandomForestRegressor()
rf_model.fit(x_train, y_train)

RandomForestRegressor()

In [20]:
pred1 = rf_model.predict(x_valid)

In [18]:
def SMAPE(y, pred):
    smape = abs((y - pred))/((abs(y) + abs(pred)) / 2) * 100
    smape = np.mean(smape)
    return smape

def mae(y, pred):
    return np.mean(abs(y-pred))

In [23]:
mae_score = mae(y_valid, pred1)
smape_score = SMAPE(y_valid, pred1)
print(f'SMAPE: {smape_score}\nMAE: {mae_score}')

SMAPE: 4.250333255678635
MAE: 81.30039592040579


In [24]:
for name, score in zip(x_train.columns, rf_model.feature_importances_):
    print(name, score)

building_number 0.879340385583798
temperature 0.004212260892793256
rainfall 0.00016481289745802342
windspeed 0.0013369682745319226
humidity 0.0012535727138395694
day 0.023225120334825353
month 0.0006960749489142835
week 0.010202415879510115
holiday 0.000510434341301627
sin_time 0.02564888170377406
cos_time 0.04731002692511713
THI 0.001999010821063659
CDH 0.004100034683072987


## Result 

시계열 특성을 고려하여 month , day , time 변수를 추가하여  
random forest 모델을 생성하여 예측한 결과 MAPE : 7.25%

building number, day , temperature time ... 순서로 중요도가 나타남  
건물 용도가 전력 수요에 가장 중요한 요소이며, 그 다음으로 일자와 온도, 시간 순으로 중요하 것을 알 수 있었음

+ update
cooling_area 냉방 면적에 대한 정보를 추가할 경우 7.22%로 mape값이 내려감
+ update 
building_type에 대한 정보를 추가한 결과 RMSE값이 감소함(318.623)

+ update
building_area 에 대한 정보를 추가할 경우 7.21%로 mape값이 내려감
그러나 RMSE가 3.19828로 상승함

## Extra Tree

In [30]:
from sklearn.tree import ExtraTreeRegressor
model5 = ExtraTreeRegressor()

In [26]:
model5.fit(x_train, y_train)

ExtraTreeRegressor()

In [27]:
pred5 = model5.predict(x_valid)

In [28]:
mae_score = mae(y_valid, pred5)
smape_score = SMAPE(y_valid, pred5)
print(f'SMAPE: {smape_score}\nMAE: {mae_score}')

SMAPE: 7.682296871712366
MAE: 154.86930291781914


In [29]:
for name, score in zip(x_train.columns, model5.feature_importances_):
    print(name, score)

building_number 0.879694052596396
temperature 0.004256220183999076
rainfall 0.0007306835841246881
windspeed 0.008629345131605584
humidity 0.007982044199765767
day 0.01695205304049795
month 0.005319535975651557
week 0.008565403950695731
holiday 0.0043416664287387885
sin_time 0.016760042979954468
cos_time 0.04024471400836533
THI 0.002825805222053525
CDH 0.0036984326981514652


## Random Forest ( modeling each building number )

In [25]:
# 각 building_number에 따라 모델을 저장할 딕셔너리 생성
models_by_building = {}

for building_num in range(1, 101):
    # 해당 building_number에 해당하는 데이터 추출
    building_data = x_train[x_train['building_number'] == building_num]
    building_target = y_train[x_train['building_number'] == building_num]
    
    # RandomForestRegressor 모델 생성 및 학습
    rf_model = RandomForestRegressor()
    rf_model.fit(building_data, building_target)
    
    # 모델을 딕셔너리에 저장
    models_by_building[building_num] = rf_model

In [28]:
# 각 building_number에 따라 예측하고 SMAPE와 MAE 값을 계산하여 저장할 리스트 생성
smape_scores = []
mae_scores = []

for building_num in range(1, 101):
    # 해당 building_number에 해당하는 검증 데이터 추출
    building_data_valid = x_valid[x_valid['building_number'] == building_num]
    building_target_valid = y_valid[x_valid['building_number'] == building_num]
    
    # 해당 building_number에 해당하는 model select
    target_model = models_by_building.get(building_num)
    if target_model is None:
        continue
    
    # prediction
    predictions = target_model.predict(building_data_valid)
    
    # SMAPE 계산
    smape_score = SMAPE(building_target_valid, predictions)
    smape_scores.append(smape_score)
    
    # MAE 계산
    mae_score = mean_absolute_error(building_target_valid, predictions)
    mae_scores.append(mae_score)

# SMAPE와 MAE 값의 평균 계산
average_smape = np.mean(smape_scores)
average_mae = np.mean(mae_scores)

print(f"SMAPE: {average_smape:.2f}%")
print(f"MAE: {average_mae:.2f}")


SMAPE: 4.18%
MAE: 79.73


## Extra Tree ( modeling by building number)

In [31]:
# 각 building_number에 따라 모델을 저장할 딕셔너리 생성
ex_models_by_building = {}

for building_num in range(1, 101):
    # 해당 building_number에 해당하는 데이터 추출
    building_data = x_train[x_train['building_number'] == building_num]
    building_target = y_train[x_train['building_number'] == building_num]
    
    # RandomForestRegressor 모델 생성 및 학습
    ex_model = ExtraTreeRegressor()
    ex_model.fit(building_data, building_target)
    
    # 모델을 딕셔너리에 저장
    ex_models_by_building[building_num] = ex_model

In [32]:
# 각 building_number에 따라 예측하고 SMAPE와 MAE 값을 계산하여 저장할 리스트 생성
smape_scores = []
mae_scores = []

for building_num in range(1, 101):
    # 해당 building_number에 해당하는 검증 데이터 추출
    building_data_valid = x_valid[x_valid['building_number'] == building_num]
    building_target_valid = y_valid[x_valid['building_number'] == building_num]
    
    # 해당 building_number에 해당하는 model select
    target_model = ex_models_by_building.get(building_num)
    if target_model is None:
        continue
    
    # prediction
    predictions = target_model.predict(building_data_valid)
    
    # SMAPE 계산
    smape_score = SMAPE(building_target_valid, predictions)
    smape_scores.append(smape_score)
    
    # MAE 계산
    mae_score = mean_absolute_error(building_target_valid, predictions)
    mae_scores.append(mae_score)

# SMAPE와 MAE 값의 평균 계산
average_smape = np.mean(smape_scores)
average_mae = np.mean(mae_scores)

print(f"SMAPE: {average_smape:.2f}%")
print(f"MAE: {average_mae:.2f}")

SMAPE: 5.82%
MAE: 112.07


## Submission

In [34]:
test_df = test_df.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
test_df.drop('num_date_time', axis = 1, inplace=True)

In [35]:
test_df = pd.merge(test_df, building_info, on='building_number', how='left')

In [36]:
test_df = test_df.drop(['solar_power_capacity', 'ess_capacity', 'pcs_capacity'], axis=1)

In [37]:
test_df[columns_to_fill] = test_df[columns_to_fill].fillna(0)

In [38]:
# 시계열 변수 생성
date = pd.to_datetime(test_df.date_time)
test_df['hour'] = date.dt.hour
test_df['day'] = date.dt.weekday
test_df['month'] = date.dt.month
test_df['week'] = date.dt.weekofyear

In [39]:
test_df['building_type_numeric'] = test_df['building_type'].map(building_type_mapping)

In [40]:
### 공휴일 변수 추가
test_df['holiday'] = test_df['day'].apply(lambda x: 1 if x >= 6 else 0)

In [41]:
## https://dacon.io/competitions/official/235680/codeshare/2366?page=1&dtype=recent
# 순환적 성격의 시간 변수 생성
test_df['sin_time'] = np.sin(2*np.pi*test_df.hour/24)
test_df['cos_time'] = np.cos(2*np.pi*test_df.hour/24)

In [42]:
## https://dacon.io/competitions/official/235736/codeshare/2743?page=1&dtype=recent
# 상대습도 변수 생성
test_df['THI'] = 9/5*test_df['temperature'] - 0.55*(1-test_df['humidity']/100)*(9/5*test_df['humidity']-26)+32

In [43]:
# Cooling Degree Hour 변수 생성
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

cdhs = np.array([])
for num in range(1,101,1):
    temp = test_df[test_df['building_number'] == num]
    cdh = CDH(temp['temperature'].values)
    cdhs = np.concatenate([cdhs, cdh])
test_df['CDH'] = cdhs

In [44]:
x_test = test_df.drop(columns =  ['date_time','building_type',
                                  'total_area','cooling_area',
                                   'hour','building_type_numeric'])

In [54]:
len(x_test)

16800

In [58]:
submission = pd.read_csv("/Users/idoyun/Desktop/데이터 분석 경진대회/진행중/2023 전력사용량 예측 AI 경진대회/datafile/sample_submission.csv")

# 예측값을 저장할 리스트 생성
predicted_answers = []

for building_num in range(1, 101):
    # 해당 building_number에 해당하는 모델 선택
    target_model = models_by_building.get(building_num)
    if target_model is None:
        # 해당 building_number에 모델이 없으면 24개의 0으로 예측값 추가
        predicted_answers.extend([0] * 24)
    else:
        # 해당 모델로 예측 수행
        predictions = target_model.predict(x_test[x_test['building_number'] == building_num])
        predicted_answers.extend(predictions)

# 예측값을 submission 데이터프레임의 'answer' 컬럼에 대입
submission['answer'] = predicted_answers

In [60]:
submission.head()

,num_date_time,answer
0,1_20220825 00,2030.9232
1,1_20220825 01,2170.0560
2,1_20220825 02,1900.8576
3,1_20220825 03,1928.8176
4,1_20220825 04,1943.6784


In [61]:
submission.to_csv('/Users/idoyun/Desktop/데이터 분석 경진대회/진행중/2023 전력사용량 예측 AI 경진대회/datafile/baseline_submission_rf_each.csv', index=False)

In [62]:
submission

,num_date_time,answer
0,1_20220825 00,2030.9232
1,1_20220825 01,2170.0560
2,1_20220825 02,1900.8576
3,1_20220825 03,1928.8176
4,1_20220825 04,1943.6784
...,...,...
16795,100_20220831 19,860.4504
16796,100_20220831 20,812.7720
16797,100_20220831 21,754.8504
16798,100_20220831 22,707.5584
